### Verificar que se obtiene bien el NDCG

In [75]:
import os
import pandas as pd
import tensorflow as tf
from src.experiments.Common import load_best_model

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

datasets = {"restaurants":["gijon", "barcelona", "madrid", "paris", "newyorkcity"],
            "pois":["barcelona", "madrid", "paris", "newyorkcity", "london"],
            "amazon":["fashion", "digital_music"]}

models = ["ATT2ITM", "BOW2ITM", "USEM2ITM"]

In [76]:
model = models[0]
dataset = "restaurants"
subset = "gijon"

# Cargar el modelo con mejores hiperparámetros
model_class = load_best_model(model=model, dataset=dataset, subset=subset)
model_class.train(dev=True, save_model=True) # Cargar el modelo

Loading best ATT2ITM model: 544d98b3ba0a0dfd7e3f0019f4d9675e
[WARNING] Model folder already exists...
[WARNING] Model weights are not loaded!
[INFO] Model already trained. Loading weights...


In [77]:
train_dev_data = model_class.DATASET.DATA["TRAIN_DEV"]
dev_data = train_dev_data[train_dev_data["dev"] == 1]

all_data = pd.read_pickle(model_class.DATASET.DATASET_PATH+"ALL_DATA")[["reviewId", "text"]]
dev_text_data = all_data.set_index("reviewId").loc[dev_data.reviewId.values]["text"].values
dev_sequence = model_class.__create_tfdata__(dev_data)

predictions = model_class.MODEL.predict(dev_sequence.batch(512).prefetch(tf.data.AUTOTUNE), verbose=0)
evaluation = model_class.evaluate(test=False)

[INFO] There are 3935 evaluation examples.
       loss    NDCG@1   NDCG@10   NDCG@50  NDCG@100   NDCG@-1  Precision@-1  \
0  0.011254  0.390343  0.552709  0.597387   0.60616  0.608079      0.589127   

   Precision@1  Precision@5  Precision@10  Recall@-1  Recall@1  Recall@5  \
0     0.390343     0.126963      0.073571    0.16798  0.390343  0.634816   

   Recall@10  Recall@20  Recall@50    F1@-1      F1@1      F1@5     F1@10  
0   0.735705   0.833799   0.933164  0.26142  0.390343  0.211605  0.133765  


In [79]:
from sklearn.metrics import ndcg_score
import numpy as np

ndcgs = []
ndcgs_at_k = []

for idx, (data_x, data_y) in enumerate(dev_sequence):
    real_y = data_y.numpy()
    pred_y = predictions[idx]
    
    real_y = np.asarray([real_y])
    pred_y = np.asarray([pred_y])
    
    ndcg = ndcg_score(real_y, pred_y)   
    ndcgs.append(ndcg)

    ndcg_at_k = ndcg_score(real_y, pred_y, k=10)
    ndcgs_at_k.append(ndcg_at_k)

In [80]:
print(np.mean(ndcgs), np.mean(ndcgs_at_k))

0.6080785941082923 0.5527091085915189
